In [3]:
# 단층 퍼셉트론 - AND_gate
def AND_gate(x1, x2):
    w1=0.5
    w2=0.5
    b=-0.7
    result = x1*w1 + x2*w2 + b
    if result <= 0:
        return 0
    else:
        return 1
    
AND_gate(0, 0), AND_gate(0, 1), AND_gate(1, 0), AND_gate(1, 1)

(0, 0, 0, 1)

In [5]:
# 단층 퍼셉트론 - NAND_gate
def NAND_gate(x1, x2):
    w1=-0.5
    w2=-0.5
    b=+0.7
    result = x1*w1 + x2*w2 + b
    if result <= 0:
        return 0
    else:
        return 1
    
NAND_gate(0, 0), NAND_gate(0, 1), NAND_gate(1, 0), NAND_gate(1, 1)

(1, 1, 1, 0)

In [6]:
# 단층 퍼셉트론 - OR_gate
def OR_gate(x1, x2):
    w1=0.6
    w2=0.6
    b=-0.5
    result = x1*w1 + x2*w2 + b
    if result <= 0:
        return 0
    else:
        return 1
    
OR_gate(0, 0), OR_gate(0, 1), OR_gate(1, 0), OR_gate(1, 1)

(0, 1, 1, 1)

In [15]:
# 단층 퍼셉트론 - XOR_gate
# '직선 1개'로 '2개 영역'을 나눌 수 없으므로 불가능하다. 그래도 도전

# 모듈 불러오기
import torch

# GPU 연산 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# 데이터 정의(입력, 출력)
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

# 단층 퍼셉트론
# 입력 + 은닉 + 출력층 개수, 활성화 함수(시그모이드 함수) 정의
linear = torch.nn.Linear(2, 1, bias=True)
sigmoid = torch.nn.Sigmoid() # 활성화 함수 : 시그모이드 함수
model = torch.nn.Sequential(linear, sigmoid).to(device)

# 비용함수(바이너 크로스 엔트로피 함수), 옵티마이저(확률적 경사하강법) 정의
criterion = torch.nn.BCELoss().to(device) # 0 or 1을 예측하는 이진분류 문제 >> 비용 함수 : 바이너리 크로스엔트로피 함수
optimizer = torch.optim.SGD(model.parameters(), lr=1) # 옵티마이저 : 확률적 경사하강법

# 학습(10001회)
for step in range(10001):
    # 옵티마이저 초기화
    optimizer.zero_grad()
    
    # 학습
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()
    
    # 출력(cost)
    if step % 100 == 0:
        print(step, cost.item())

# 예측값
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('모델의 출력값(Hypothesis): ', hypothesis.detach().cpu().numpy())
    print('모델의 예측값(Predicted): ', predicted.detach().cpu().numpy())
    print('실제값(Y): ', Y.cpu().numpy())
    print('정확도(Accuracy): ', accuracy.item())

0 0.7273974418640137
100 0.6931476593017578
200 0.6931471824645996
300 0.6931471824645996
400 0.6931471824645996
500 0.6931471824645996
600 0.6931471824645996
700 0.6931471824645996
800 0.6931471824645996
900 0.6931471824645996
1000 0.6931471824645996
1100 0.6931471824645996
1200 0.6931471824645996
1300 0.6931471824645996
1400 0.6931471824645996
1500 0.6931471824645996
1600 0.6931471824645996
1700 0.6931471824645996
1800 0.6931471824645996
1900 0.6931471824645996
2000 0.6931471824645996
2100 0.6931471824645996
2200 0.6931471824645996
2300 0.6931471824645996
2400 0.6931471824645996
2500 0.6931471824645996
2600 0.6931471824645996
2700 0.6931471824645996
2800 0.6931471824645996
2900 0.6931471824645996
3000 0.6931471824645996
3100 0.6931471824645996
3200 0.6931471824645996
3300 0.6931471824645996
3400 0.6931471824645996
3500 0.6931471824645996
3600 0.6931471824645996
3700 0.6931471824645996
3800 0.6931471824645996
3900 0.6931471824645996
4000 0.6931471824645996
4100 0.6931471824645996
4200

In [22]:
# 다층 퍼셉트론

# 모듈 불러오기
import torch
import torch.nn as nn

# GPU 연산 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
    
# 데이터 정의(입력, 출력)
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

# 단층 퍼셉트론
# 입력 + 은닉 + 출력층 개수, 활성화 함수(시그모이드 함수) 정의
model = nn.Sequential(
    nn.Linear(2, 10, bias=True), nn.Sigmoid(), # 입력층 = 2, 은닉층 = 10
    nn.Linear(10, 10, bias=True), nn.Sigmoid(), # 은닉층 = 10, 은닉층 = 10
    nn.Linear(10, 10, bias=True), nn.Sigmoid(), # 은닉층 = 10, 은닉층 = 10
    nn.Linear(10, 1, bias=True), nn.Sigmoid()).to(device) # 은닉층 = 10, 출력층 = 1

# 비용함수(바이너 크로스 엔트로피 함수), 옵티마이저(확률적 경사하강법) 정의
criterion = torch.nn.BCELoss().to(device) # 0 or 1을 예측하는 이진분류 문제 >> 비용 함수 : 바이너리 크로스엔트로피 함수
optimizer = torch.optim.SGD(model.parameters(), lr=1) # 옵티마이저 : 확률적 경사하강법

# 학습(10001회)
for epoch in range(10001):
    # 옵티마이저 초기화
    optimizer.zero_grad()
    
    # 학습
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()
    
    # 출력(cost)
    if epoch % 100 == 0:
        print(epoch, cost.item()) 
        # 엄청 작은 데이터 똑같은것만 10001번 반복하는데 5000번이 넘어서야 cost가 조금씩 줄어드네? 뭐지?

# 예측값
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('모델의 출력값(Hypothesis): ', hypothesis.detach().cpu().numpy())
    print('모델의 예측값(Predicted): ', predicted.detach().cpu().numpy())
    print('실제값(Y): ', Y.cpu().numpy())
    print('정확도(Accuracy): ', accuracy.item())

0 0.6948983669281006
100 0.693155825138092
200 0.6931535601615906
300 0.6931513547897339
400 0.693149209022522
500 0.6931473016738892
600 0.6931453943252563
700 0.6931434273719788
800 0.6931416988372803
900 0.6931397914886475
1000 0.6931380033493042
1100 0.6931361556053162
1200 0.6931343078613281
1300 0.6931324005126953
1400 0.6931304931640625
1500 0.6931284666061401
1600 0.6931264400482178
1700 0.6931242942810059
1800 0.6931220889091492
1900 0.6931197047233582
2000 0.6931171417236328
2100 0.6931145191192627
2200 0.6931115984916687
2300 0.6931084990501404
2400 0.6931051015853882
2500 0.6931014657020569
2600 0.6930974721908569
2700 0.6930930018424988
2800 0.6930879950523376
2900 0.6930825114250183
3000 0.6930763721466064
3100 0.6930692195892334
3200 0.6930612325668335
3300 0.6930519342422485
3400 0.6930410861968994
3500 0.6930283904075623
3600 0.6930132508277893
3700 0.6929951310157776
3800 0.6929728984832764
3900 0.6929453015327454
4000 0.6929103136062622
4100 0.6928649544715881
4200 0